In [ ]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# workshop folder, e.g. 'acmlab/workshops/project'
FOLDERNAME = 'primary_description'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/My Drive/primary_description


In [ ]:
import numpy as np
import os
from sklearn.metrics import f1_score
import pandas as pd
from textblob import TextBlob
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet

from IPython.display import display

# import stanza
# stanza.install_corenlp()
# import spacy
# nltk.download('brown')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
data = pd.read_json("Practice/cs.ai-ann0.json")
# text_data = data.loc["text"]
# text_data.to_dict()

In [ ]:
def extract_primary_description(entity): #Entity is a dicitionary.
  output = {}
  for key in entity.keys():
    if entity[key]['label'] == "PRIMARY":
      output[key] = entity[key]
  return output


In [ ]:
def compute(text): #This function will take text and output a data structure of primary descriptions that are predicted
  
  
  
  
  # is_noun = lambda pos: pos[:2] == 'NN'
  # # do the nlp stuff
  # tokenized = nltk.word_tokenize(text)
  # nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)] 
  # print(nouns)
  
  # blob = TextBlob(text)
  # print(blob.noun_phrases)
  return text

In [ ]:
lemmatizer = nltk.WordNetLemmatizer()
def setup(text):

  #word tokenizeing and part-of-speech tagger
  document = text
  tokens = [nltk.word_tokenize(sent) for sent in [document]]
  postag = [nltk.pos_tag(sent) for sent in tokens][0]

  # Rule for NP chunk and VB Chunk
  grammar = r"""
      NBAR:
          {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns
          {<RB.?>*<VB.?>*<JJ>*<VB.?>+<VB>?} # Verbs and Verb Phrases
          
      NP:
          {<NBAR>}
          {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
          
  """
  #Chunking
  cp = nltk.RegexpParser(grammar)

  # the result is a tree
  tree = cp.parse(postag)
  return tree

def leaves(tree):
    """Finds NP (nounphrase) leaf nodes of a chunk tree."""
    for subtree in tree.subtrees(filter = lambda t: t.label() =='NP'):
        yield subtree.leaves()
        
def get_word_postag(word):
    if pos_tag([word])[0][1].startswith('J'):
        return wordnet.ADJ
    if pos_tag([word])[0][1].startswith('V'):
        return wordnet.VERB
    if pos_tag([word])[0][1].startswith('N'):
        return wordnet.NOUN
    else:
        return wordnet.NOUN
    
def normalise(word):
    """Normalises words to lowercase and stems and lemmatizes it."""
    # word = word.lower()
    # postag = get_word_postag(word)
    # word = lemmatizer.lemmatize(word, postag)
    return word

def get_terms(tree):    
    for leaf in leaves(tree):
        terms = [normalise(w) for w,t in leaf]
        yield terms

# def getNounPhrases(text):
#   tree = setup(text)

#   terms = get_terms(tree)

#   features = []
#   for term in terms:
#       _term = ''
#       for word in term:
#           _term += ' ' + word
#       if '\\' not in _term:
#         features.append(_term.strip())
#   return features

def getNounPhrases(text):
  wordsToRemove = ['be', 'is', 'are', 'was', 'were', 'been', 'being']
  tree = setup(text)

  terms = get_terms(tree)

  features = []
  for term in terms:
      _term = ''
      for word in term:
        _term += ' ' + word
      # if wordsToRemove not in _term:
      # if not any(_term for x in wordsToRemove):
        # print(_term)
      if not any(x in _term.split() for x in wordsToRemove) and '\\' not in _term:
        features.append(_term.strip())
  return features

In [ ]:
# print(getNounPhrases("Mostly, there just is no default way of determining the paragraph boundary and people tend to work with sentences. Still, the unit of a paragraph might be of a higher value than that of a sentence. Examples might be: coreference resolutions that overlap multiple sentences. Questions that find their answer throughout a whole paragraph. A reader that understands a paragraph better than an isolated sentence. It’s clear that the signal from a writer is best expressed in a paragraph."))

In [ ]:
df = data
for column in df:
    wantedOutput = extract_primary_description(df[column]["entity"])
    paragraph = df[column]["text"]
    print("PARAGRAPH:\n", paragraph)
    print("NOUN PHRASES:\n", getNounPhrases(paragraph))
    print("EXPECTED:\n", wantedOutput)
    print("*" * 280)
    print("")

PARAGRAPH:
 In this model , the input vectors are representations of whether the student answered a particular question correctly or incorrectly at the previous time step , and the output vectors are representations of the probability , over all the questions in the question bank , that a student will get the question correct at the following time step .
In \cite{DKTNIPS} , the authors propose using a one - hot vector $\Xstvec \in \R^{2I}$ to represent the response of a student $s$ ( on item $i$ ) at time $t$ .
Here $I$ is the total number of items and the first $I$ slots represent answering correctly and the remaining $I$ slots represent answering incorrectly .
Output vectors $\Ystvec \in \R^I$ are vectors of probabilities , where the $i$ th element of $\Ystvec$ is the model 's predicted probability that student $s$ would answer item $i$ correctly at time $t + 1$ .
NOUN PHRASES:
 ['model', 'input vectors', 'representations', 'student', 'answered', 'particular question', 'previous time

In [ ]:
def findNounsWithLocs(text):
  '''This function takes in a block of text, finds the nouns in it and then returns an array of 1s and 0s representing where those nouns are'''
  originalText = text
  #modify text here however we please in getNounPhrases
  nounList = getNounPhrases(text)
  start = 0
  predicted_array = np.zeros(len(originalText))

  for word in nounList:
    nounStartLoc = originalText.find(word, start)
    nounEndLoc = nounStartLoc + len(word)

    if abs(originalText.find('$', start) - nounStartLoc) < 25:
      predicted_array[nounStartLoc : nounEndLoc] = 1

    start = nounEndLoc

  return predicted_array

In [ ]:
print(findNounsWithLocs("The bus is yellow."))

[0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
for column in data:
  paragraph = df[column]["text"]
  print("PARAGRAPH:\n", paragraph)
  print("NOUN PHRASES:\n", getNounPhrases(paragraph))
  print("NOUN LOCATIONS:\n", findNounsWithLocs(paragraph))
  print('*' * 280 + '\n')

Streaming output truncated to the last 5000 lines.
NOUN LOCATIONS:
 [0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0

In [ ]:
compute("""We defer to \\cref{sec:compstat_general} the systematic analysis of when a wishful Receiver would take an action more / less often than a Bayesian .\nHowever , we can already point out that such differences in behavior between a Bayesian and a wishful Receiver , i.e. how sets $\\Delta_a^W$ are distorted with respect to $\\Delta_a^B$ for every $a\\in A$ are systematically related to the asymmetry of the payoff matrix and will be all the more pronounced the closer the cost parameter $\\theta$ is to $0$ .\n\\end{comment}  \\begin{comment}  \\subsubsection{Investors application with            }  \\label{secap:investor_infinite_state}""")

'We defer to \\cref{sec:compstat_general} the systematic analysis of when a wishful Receiver would take an action more / less often than a Bayesian .\nHowever , we can already point out that such differences in behavior between a Bayesian and a wishful Receiver , i.e. how sets $\\Delta_a^W$ are distorted with respect to $\\Delta_a^B$ for every $a\\in A$ are systematically related to the asymmetry of the payoff matrix and will be all the more pronounced the closer the cost parameter $\\theta$ is to $0$ .\n\\end{comment}  \\begin{comment}  \\subsubsection{Investors application with            }  \\label{secap:investor_infinite_state}'

In [ ]:
def create_arrays(expected, predicted, len_text):
  expected_array = np.zeros(len_text)  
  predicted_array = np.zeros(len_text)
  for key in expected.keys():
    start_index = expected[key]["start"]
    end_index = expected[key]["end"]
    expected_array[start_index : end_index + 1] = 1

  
  # for key in predicted.keys():
  #   start_index = predicted[key]["start"]
  #   end_index = predicted[key]["end"]
  #   predicted_array[start_index : end_index + 1] = 1

  return (expected_array, predicted_array)

In [ ]:
def compare(expected, predicted): #Computes the F1 score between expected and predicted
  expected_array = np.array(len())

In [ ]:
def framework(df):
  results = []
  for column in df:
    paragraph = df[column]["text"]
    expected = df[column]["entity"]
    primary_description = extract_primary_description(expected)
    predicted = compute(paragraph)
    # expected_array, predicted_array = create_arrays(primary_description, predicted, len(paragraph))
    expected_array, predicted_array = create_arrays(primary_description, predicted, len(paragraph))
    predicted_array = findNounsWithLocs(paragraph)
    result = f1_score(expected_array, predicted_array, average= "binary", zero_division = 1)
    results.append(result)
  print("Average F1 score is:", sum(results) / len(results))


In [ ]:
#"The current F1 score of 0.355 arises when we leave the predicted_array all full of zeroes and compare with the expected."
# 0.261 is our current best implementing NLP techniques w/ <25
framework(data)

Average F1 score is: 0.2611429499175073


In [ ]:
dataFiles = []
pwd = '/content/drive/My Drive/primary_description/Practice'
for filename in os.listdir(pwd):
    if filename.endswith("json"): 
        dataFiles.append(pwd + '/' + filename)
for filename in dataFiles:
  print(filename.replace('/content/drive/My Drive/primary_description/Practice/', ""))
  framework(pd.read_json(filename))
  print()

physics.atom_ph-ann10.json
Average F1 score is: 0.24100718166495289

q_bio.qm-ann11.json
Average F1 score is: 0.24836546363431322

cs.ai-ann0.json
Average F1 score is: 0.2611429499175073

cs.ai-ann3.json
Average F1 score is: 0.292149992910222

econ.th-ann6.json
Average F1 score is: 0.18309014547466393

physics.atom_ph-ann8.json
Average F1 score is: 0.10158271561332784

econ.th-ann5.json
Average F1 score is: 0.10712998732570396

physics.atom_ph-ann9.json
Average F1 score is: 0.2622680957747418

math.co-ann7.json
Average F1 score is: 0.27230732143308856

cs.ai-ann2.json
Average F1 score is: 0.2836446512781167

econ.th-ann4.json
Average F1 score is: 0.23045668243813608

